In [1]:
import ray

In [2]:
def create_model():
    import tensorflow as tf
    print(tf.config.list_physical_devices('GPU'))
    
    @tf.function
    def model(x):
        return -1*(x*x)
    
    def grad(self,x):
        x  = tf.convert_to_tensor(x)
        with tf.GradientTape() as tape:
            tape.watch(x)
            res = model(x)
        return tape.gradient(res, [x])
    
    return model, grad

In [3]:
## this crashes the kernel
#ray.init()


In [3]:
@ray.remote(num_gpus=1)
class Simulator(object):
    def __init__(self):
        self.model, self.grad = create_model()
        
    def fwd(self,x):
        return self.model(x)
    
    def gradient(self,x):
        return self.grad(x)

In [4]:
my_sim=Simulator

In [8]:
# does not work, possibly because I never ran init()?
my_sim.gradient(0.1)
my_sim.fwd(0.1)

AttributeError: 'ActorClass(Simulator)' object has no attribute 'grad'

In [9]:
#same error...
from ray.util.multiprocessing import Pool as RayPool
import bayesfast as bf
bf.utils.parallel.set_backend(RayPool(4))
bf.utils.random.set_generator(4)
den = bf.DensityLite(logp=my_sim.model,grad=my_sim.fwd,input_size=1)
#

2020-10-20 13:07:54,597	INFO services.py:1164 -- View the Ray dashboard at http://127.0.0.1:8265


AttributeError: 'ActorClass(Simulator)' object has no attribute 'model'